# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [2]:
### General parameters
max_steps = 5e6 # Set maximum number of steps to run environment.
run_path = "ppo" # The sub-directory name for model and summary statistics
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 1000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "pong" # Name of the training environment file.
curriculum_file = "curricula/pong.json"

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 5 # Number of gradient descent steps per batch of experiences.
num_layers = 2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 2048 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 64 # Number of units in hidden layer.
batch_size = 64 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [3]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'Academy' started successfully!


Unity Academy name: Academy
        Number of brains: 1
        Reset Parameters :
		paddleScale -> 4.0
Unity brain name: Brain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 5
        Action space type: discrete
        Action space size (per agent): 3
        Memory space size (per agent): 0
        Action descriptions: , , 


### Train the Agent(s)

In [ ]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Step: 1000. Mean Reward: 0.5. Std of Reward: 0.8660254037844386.
Step: 2000. Mean Reward: -0.3333333333333333. Std of Reward: 0.9428090415820634.
Step: 3000. Mean Reward: 0.2. Std of Reward: 0.9797958971132713.
Step: 4000. Mean Reward: 0.0. Std of Reward: 1.0.
Step: 5000. Mean Reward: -0.3333333333333333. Std of Reward: 0.9428090415820634.
Step: 6000. Mean Reward: -0.2. Std of Reward: 0.9797958971132713.
Step: 7000. Mean Reward: -0.42857142857142855. Std of Reward: 0.9035079029052513.
Step: 8000. Mean Reward: -1.0. Std of Reward: 0.0.
Step: 9000. Mean Reward: 0.16666666666666666. Std of Reward: 0.8975274678557508.
Step: 10000. Mean Reward: -1.0. Std of Reward: 0.0.
Step: 11000. Mean Reward: -0.2. Std of Reward: 0.9797958971132713.
Step: 12000. Mean Reward: -0.3333333333333333. Std of Reward: 0.9428090415820634.
Step: 13000. Mean Reward: -0.3333333333333333. Std of Reward: 0.9428090415820634.
Step: 14000. Mean Reward: 0.0. Std of Reward: 1.0.
Step: 15000. Mean Reward: 0.0. Std of Reward

Step: 123000. Mean Reward: 0.0. Std of Reward: 1.0.
Step: 124000. Mean Reward: -0.2. Std of Reward: 0.9797958971132713.
Step: 125000. Mean Reward: 0.0. Std of Reward: 0.9428090415820634.
Step: 126000. Mean Reward: 0.0. Std of Reward: 1.0.
Step: 127000. Mean Reward: -1.0. Std of Reward: 0.0.
Step: 128000. Mean Reward: 0.3333333333333333. Std of Reward: 0.9428090415820634.
Step: 129000. Mean Reward: 0.3333333333333333. Std of Reward: 0.9428090415820634.
Step: 130000. Mean Reward: -0.3333333333333333. Std of Reward: 0.9428090415820634.
Step: 131000. Mean Reward: -0.6. Std of Reward: 0.8000000000000002.
Step: 132000. Mean Reward: 0.14285714285714285. Std of Reward: 0.989743318610787.
Step: 133000. Mean Reward: -0.2. Std of Reward: 0.9797958971132713.
Step: 134000. Mean Reward: 0.0. Std of Reward: 0.9258200997725514.
Step: 135000. Mean Reward: -1.0. Std of Reward: 0.0.
Step: 136000. Mean Reward: 0.0. Std of Reward: 1.0.
Step: 137000. Mean Reward: 0.3333333333333333. Std of Reward: 0.9428090

Step: 244000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 245000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 246000. Mean Reward: 0.5. Std of Reward: 0.8660254037844386.
Step: 247000. Mean Reward: 0.75. Std of Reward: 0.6614378277661477.
Step: 248000. Mean Reward: -0.5. Std of Reward: 0.8660254037844386.
Step: 249000. Mean Reward: 0.5. Std of Reward: 0.8660254037844386.
Step: 250000. Mean Reward: 0.5. Std of Reward: 0.7637626158259734.
Saved Model
Step: 251000. Mean Reward: 0.42857142857142855. Std of Reward: 0.9035079029052513.
Step: 252000. Mean Reward: 0.0. Std of Reward: 1.0.
Step: 253000. Mean Reward: 0.3333333333333333. Std of Reward: 0.9428090415820634.
Step: 254000. Mean Reward: -0.3333333333333333. Std of Reward: 0.9428090415820634.
Step: 255000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 256000. Mean Reward: 0.0. Std of Reward: 1.0.
Step: 257000. Mean Reward: -0.6. Std of Reward: 0.8.
Step: 258000. Mean Reward: 0.7142857142857143. Std of Reward: 0.6998542122237652.
Step: 25

INFO:unityagents:
Lesson changed. Now in Lesson 1 : 	paddleScale -> 8


Step: 275000. Mean Reward: 0.8571428571428571. Std of Reward: 0.3499271061118826.
Step: 276000. Mean Reward: 0.6666666666666666. Std of Reward: 0.7453559924999298.
Step: 277000. Mean Reward: 0.7142857142857143. Std of Reward: 0.6998542122237652.
Step: 278000. Mean Reward: 0.5. Std of Reward: 0.8660254037844386.
Step: 279000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 280000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 281000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 282000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 283000. Mean Reward: 0.3333333333333333. Std of Reward: 0.9428090415820634.
Step: 284000. Mean Reward: 0.75. Std of Reward: 0.4330127018922193.
Step: 285000. Mean Reward: 0.2. Std of Reward: 0.9797958971132713.
Step: 286000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 287000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 288000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 289000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 290000. Mean Reward: 0.42857142857142855. Std of

...

Step: 1230000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1231000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1232000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1233000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1234000. Mean Reward: 0.5714285714285714. Std of Reward: 0.7284313590846836.
Step: 1235000. Mean Reward: 0.6. Std of Reward: 0.8.
Step: 1236000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1237000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1238000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1239000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1240000. Mean Reward: 0.6666666666666666. Std of Reward: 0.7453559924999298.
Step: 1241000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1242000. Mean Reward: 0.6666666666666666. Std of Reward: 0.4714045207910317.
Step: 1243000. Mean Reward: 0.6. Std of Reward: 0.8.
Step: 1244000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1245000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1246000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 124

Step: 1361000. Mean Reward: 0.5. Std of Reward: 0.8660254037844386.
Step: 1362000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1363000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1364000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1365000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1366000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1367000. Mean Reward: 0.375. Std of Reward: 0.8569568250501305.
Step: 1368000. Mean Reward: 0.5. Std of Reward: 0.8660254037844386.
Step: 1369000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1370000. Mean Reward: 0.2. Std of Reward: 0.9797958971132713.
Step: 1371000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1372000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1373000. Mean Reward: 0.7142857142857143. Std of Reward: 0.6998542122237652.
Step: 1374000. Mean Reward: 0.7142857142857143. Std of Reward: 0.6998542122237652.
Step: 1375000. Mean Reward: 0.4. Std of Reward: 0.8.
Step: 1376000. Mean Reward: 0.7142857142857143. Std of Reward: 0.6998542122237652.


Step: 1491000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1492000. Mean Reward: 0.6666666666666666. Std of Reward: 0.4714045207910317.
Step: 1493000. Mean Reward: 0.6. Std of Reward: 0.8000000000000002.
Step: 1494000. Mean Reward: 0.6666666666666666. Std of Reward: 0.7453559924999298.
Step: 1495000. Mean Reward: 0.6666666666666666. Std of Reward: 0.7453559924999298.
Step: 1496000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1497000. Mean Reward: 0.3333333333333333. Std of Reward: 0.9428090415820634.
Step: 1498000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1499000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1500000. Mean Reward: 0.8. Std of Reward: 0.4.
Saved Model
Step: 1501000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1502000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1503000. Mean Reward: 0.7142857142857143. Std of Reward: 0.6998542122237652.
Step: 1504000. Mean Reward: 0.14285714285714285. Std of Reward: 0.989743318610787.
Step: 1505000. Mean Reward: 1.0. Std of Reward: 0.0

Step: 1611000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1612000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1613000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1614000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1615000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1616000. Mean Reward: 0.6666666666666666. Std of Reward: 0.7453559924999298.
Step: 1617000. Mean Reward: 0.8571428571428571. Std of Reward: 0.3499271061118826.
Step: 1618000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1619000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1620000. Mean Reward: 0.2. Std of Reward: 0.9797958971132713.
Step: 1621000. Mean Reward: 0.6666666666666666. Std of Reward: 0.7453559924999298.
Step: 1622000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1623000. Mean Reward: 0.2. Std of Reward: 0.9797958971132713.
Step: 1624000. Mean Reward: 0.7142857142857143. Std of Reward: 0.6998542122237653.
Step: 1625000. Mean Reward: 0.75. Std of Reward: 0.4330127018922193.
Step: 1626000. Mean Reward: -0.2. Std o

Step: 1733000. Mean Reward: 0.42857142857142855. Std of Reward: 0.9035079029052513.
Step: 1734000. Mean Reward: 0.8571428571428571. Std of Reward: 0.3499271061118826.
Step: 1735000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1736000. Mean Reward: 0.42857142857142855. Std of Reward: 0.9035079029052513.
Step: 1737000. Mean Reward: 0.42857142857142855. Std of Reward: 0.9035079029052513.
Step: 1738000. Mean Reward: 1.0. Std of Reward: 0.0.
Step: 1739000. Mean Reward: -0.14285714285714285. Std of Reward: 0.989743318610787.
Step: 1740000. Mean Reward: 0.6. Std of Reward: 0.8.
Step: 1741000. Mean Reward: 0.5. Std of Reward: 0.8660254037844386.
Step: 1742000. Mean Reward: 0.0. Std of Reward: 0.8944271909999159.


### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [6]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/ppo\model-3250000.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo\model-3250000.cptk


INFO:tensorflow:Froze 4 variables.


INFO:tensorflow:Froze 4 variables.


Converted 4 variables to const ops.
